In [1]:
def test_integral_query():
    from astropy.coordinates import SkyCoord
    import numpy as np
    from datetime import datetime
    import pandas as pd
    from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
    
    p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius="10 degree")

    cat = IntegralQuery(searchquerry)
    f = Filter(SCW_TYPE="POINTING")
    scw_ids = cat.apply_filter(f,return_coordinates=True, remove_duplicates=False)
        
    assert scw_ids.shape == (8087, 4), "The number of pointings found is not what we expect"
    
    assert np.array_equal(scw_ids[3], np.array(['003900020030', 83.635201, 22.012722,
                                                datetime(2003, 2, 7, 7, 56, 16)], dtype=object)), \
        "A content of scw_ids is not what we expect"
        
    cat.sort_by("DEC_X")
    
    assert np.array_equal(np.array(cat.table.iloc[1114]),
                          np.array(['115900130010', 1, 'POINTING', 82.235039, 16.914862, 
                                    pd.Timestamp('2012-04-10 11:09:23'), pd.Timestamp('2012-04-10 11:44:04'), 
                                    '88602500001', 'CALIBRATION', 'PUBLIC', 'PUBLIC', 2053.0, 2038.0, 
                                    2051.0, 2053.0, 2053.0, 2053.0, 2052.0, 81747968.0, 316.022])), \
                                        "Sort_by did not produce the correct table"
                                        
    f = Filter(RA_X=Range(78., None),
           DEC_X=Range(None, 30.),
           TIME=Range(min_val='2009-11-04T00:05:23', max_val='2015-09-04T00:05:23'),
           OBS_TYPE="CALIBRATION",
           PS="PUBLIC",
           PI_NAME="PUBLIC",
           GOOD_SPI=Range(None, 500))
    
    cat.sort_by("DSIZE")
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert (scw_ids.shape == (1511,)) and (scw_ids[1000]=='132700450021'), \
        "Filter gave wrong results"
                                        
    searchquerry = SearchQuery(object_name="Cyg X-1")
    cat = IntegralQuery(searchquerry)
    f = Filter()
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (19151,), "Object Query gave wrong number of pointings"
    
    f = Filter(SCW_ID='004000100010',
               SCW_VER=2)
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (0,), "SCW_ID or SCW_VER filter gave wrong result"
    
                                
test_integral_query()

In [2]:
def test_clustering():
    import numpy as np
    from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
    from IntegralPointingClustering import ClusteredQuery
    
    searchquerry = SearchQuery(object_name="Cyg X-1")
    cat = IntegralQuery(searchquerry)
    f = Filter(SCW_TYPE="POINTING")
    scw_ids = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)
    
    np.random.seed(0)
    
    cq = ClusteredQuery(scw_ids, 1/6, 1/15, 1, 0.3,
                        cluster_size_range = (5,5),
                        cluster_size_preference_threshold = (),
                        failed_improvements_max = 2,
                        suboptimal_cluster_size = 2,
                        close_suboptimal_cluster_size = 3,
                        track_performance = False)
    
    region_sizes = [len(cq.clusters[i+1]) for i in range(5)]
    
    assert region_sizes == [440, 123, 56, 73, 1710], "Cluster Sizes are not what we expect"
    
    clustered = set()
    n = 0
    for key, value in cq.clusters.items():
        for c in value:
            clustered = clustered | set(c.indices)
            n += c.num_pointings
            
    missing = [i for i in range(cq._num_pointings) if i not in clustered]
    
    assert missing == [], "Not all pointings were clustered"
    
    assert cq._num_pointings == n, "Some pointings are in multiple clusters"
    
    ss = cq.get_clustered_scw_ids()
    
    assert (ss[1][0], ss[5][-1]) == (['001130001060'], 
                                     ['250300020010',
                                      '250300090010',
                                      '250300100010',
                                      '250300110010',
                                      '250300120010']), \
                                        "The clusters are not what we expect"
    
    np.random.seed(0)
                                
    cq = ClusteredQuery(scw_ids, 0, 1/15, 1, 0.3, 6,
                    cluster_size_range = (3,6),
                    cluster_size_preference_threshold = (10.,8.,3.),
                    failed_improvements_max = 2,
                    suboptimal_cluster_size = 3,
                    close_suboptimal_cluster_size = 3,
                    track_performance = False)
    
    region_sizes = [len(cq.clusters[i+1]) for i in range(6)]
    
    assert region_sizes == [390, 136, 81, 97, 93, 1323], "Cluster Sizes are not what we expect"
    
    clustered = set()
    n = 0
    for key, value in cq.clusters.items():
        for c in value:
            clustered = clustered | set(c.indices)
            n += c.num_pointings
            
    missing = [i for i in range(cq._num_pointings) if i not in clustered]
    
    assert missing == [], "Not all pointings were clustered"
    
    assert cq._num_pointings == n, "Some pointings are in multiple clusters"
    
    ss = cq.get_clustered_scw_ids()
    
    assert (ss[1][-1], ss[6][1]) == (['240100460010'],
                                     ['001700110010',
                                      '001900050010',
                                      '001900060010',
                                      '001900070010',
                                      '001900130010',
                                      '001900140010']), \
                                        "The clusters are not what we expect"
    
    
test_clustering()

In [1]:
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
from astropy.coordinates import SkyCoord

radius_around_crab = 3

p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
searchquerry = SearchQuery(position=p, radius=f"{radius_around_crab} degree")
cat = IntegralQuery(searchquerry)

f1 = Filter(SCW_TYPE="POINTING",
            TIME=Range("2011-01-01T00:00:00", "2012-01-01T00:00:00"))

scw_ids1 = cat.apply_filter(f1, return_coordinates=True, remove_duplicates=True)
scw_ids1

array([['101900600010', 81.028214, 21.510889,
        datetime.datetime(2011, 2, 17, 19, 39, 19)],
       ['101900640010', 83.982414, 19.715889,
        datetime.datetime(2011, 2, 17, 21, 50, 16)],
       ['101900650010', 83.485291, 22.001888,
        datetime.datetime(2011, 2, 17, 22, 22, 56)],
       ['101900660010', 82.971581, 24.286083,
        datetime.datetime(2011, 2, 17, 22, 55, 37)],
       ['101900700010', 85.960335, 22.443501,
        datetime.datetime(2011, 2, 18, 1, 6, 54)],
       ['101900780010', 83.490463, 22.067528,
        datetime.datetime(2011, 2, 18, 5, 32)],
       ['101900790010', 83.495041, 22.067223,
        datetime.datetime(2011, 2, 18, 5, 44, 20)],
       ['101900790020', 83.495041, 22.067223,
        datetime.datetime(2011, 2, 18, 6, 43, 10)],
       ['101900800010', 83.494583, 22.067249,
        datetime.datetime(2011, 2, 18, 7, 25, 49)],
       ['101900810010', 83.495041, 22.067112,
        datetime.datetime(2011, 2, 18, 7, 44)],
       ['101900810020', 8

In [2]:
f2 = Filter(SCW_TYPE="POINTING",
            TIME=Range("2011-01-01T00:00:00", "2012-01-01T00:00:00"),
            TIME_ELAPSED=Range(1000, None))
scw_ids2 = cat.apply_filter(f2, return_coordinates=True, remove_duplicates=True)
scw_ids2


            SCW_ID  SCW_VER  SCW_TYPE       RA_X      DEC_X  \
1285  101900600010        1  POINTING  81.028214  21.510889   
1287  101900640010        1  POINTING  83.982414  19.715889   
1289  101900650010        1  POINTING  83.485291  22.001888   
1291  101900660010        1  POINTING  82.971581  24.286083   
1293  101900700010        1  POINTING  85.960335  22.443501   
1296  101900780010        1  POINTING  83.490463  22.067528   
1298  101900790010        1  POINTING  83.495041  22.067223   
1299  101900790020        1  POINTING  83.495041  22.067223   
1301  101900800010        1  POINTING  83.494583  22.067249   
1303  101900810010        1  POINTING  83.495041  22.067112   
1304  101900810020        1  POINTING  83.495041  22.067112   
1306  101900820010        1  POINTING  83.494667  22.067028   
1308  101900830010        1  POINTING  83.495041  22.067167   
1309  101900830020        1  POINTING  83.495041  22.067167   
1311  101900840010        1  POINTING  83.494545  22.06

array([['101900600010', 81.028214, 21.510889,
        datetime.datetime(2011, 2, 17, 19, 39, 19)],
       ['101900640010', 83.982414, 19.715889,
        datetime.datetime(2011, 2, 17, 21, 50, 16)],
       ['101900650010', 83.485291, 22.001888,
        datetime.datetime(2011, 2, 17, 22, 22, 56)],
       ['101900660010', 82.971581, 24.286083,
        datetime.datetime(2011, 2, 17, 22, 55, 37)],
       ['101900700010', 85.960335, 22.443501,
        datetime.datetime(2011, 2, 18, 1, 6, 54)],
       ['101900790010', 83.495041, 22.067223,
        datetime.datetime(2011, 2, 18, 5, 44, 20)],
       ['101900790020', 83.495041, 22.067223,
        datetime.datetime(2011, 2, 18, 6, 43, 10)],
       ['101900800010', 83.494583, 22.067249,
        datetime.datetime(2011, 2, 18, 7, 25, 49)],
       ['101900810010', 83.495041, 22.067112,
        datetime.datetime(2011, 2, 18, 7, 44)],
       ['101900810020', 83.495041, 22.067112,
        datetime.datetime(2011, 2, 18, 8, 42, 54)],
       ['101900820010